In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Veri Yükleme
data = pd.read_csv('data/apache/data.csv')

# Veri çerçevesindeki sütun adlarını kontrol et
print("Sütun Adları:", data.columns)

# Tarih sütununu datetime formatına dönüştürme
data['Date'] = pd.to_datetime(data['Date'].str.strip('[]'), format='%d/%b/%Y:%H:%M:%S %z', errors='coerce')

# Eksik Verilerin Kontrolü
missing_values = data.isnull().sum()
print("Eksik Veriler:", missing_values)

# Gereksiz Sütunları Kaldırma (eğer veri çerçevesinde varsa)
data = data.drop(columns=['Referrer', 'User Agent'], errors='ignore')

# Yeni Özellikler Ekleme: Yıl, Ay, Gün, Saat, Dakika
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data['Hour'] = data['Date'].dt.hour
data['Minute'] = data['Date'].dt.minute

# 'Status Code' sütununu sayısal değerlere dönüştürme (eğer gerekliyse)
data['Status Code'] = pd.to_numeric(data['Status Code'], errors='coerce')

# Boş veya Hatalı Verileri Temizleme
data = data.dropna(subset=['Request', 'Endpoint', 'Status Code'])
data = data[data['Request'].str.strip() != '']
data = data[data['Endpoint'].str.strip() != '']
data = data[data['Status Code'].notna()]  # Sayısal sütunları kontrol et

# Birleştirilen metin verisini oluştur
data['Combined'] = data['Request'] + ' ' + data['Endpoint'] + ' ' + data['Status Code'].astype(str)

# Boş verileri kontrol et
print(data['Combined'].head())
print(data['Combined'].isnull().sum())

# TF-IDF Vektörizasyonu
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['Combined']).toarray()

# FAISS İndeksi Oluşturma ve Vektörlerin Eklenmesi
index = faiss.IndexFlatL2(X.shape[1])
index.add(X.astype('float32'))

# Kullanıcı Sorgusunu Vektörleştirme ve En Yakın Komşuları Bulma
def find_relevant_logs(query):
    query_vector = vectorizer.transform([query]).toarray().astype('float32')
    D, I = index.search(query_vector, 5)
    return data.iloc[I[0]]

# GPT-2 Modeli ile Yanıt Oluşturma
def generate_response(logs):
    input_text = " ".join(
        logs.apply(lambda row: f"{row['Date']} {row['Request']} {row['Endpoint']} {row['Status Code']}",
                   axis=1).tolist())
    input_text = input_text[:1000]  # Giriş metnini 1000 karakterle sınırlama

    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

    output = model.generate(input_ids, attention_mask=attention_mask, max_length=150, num_return_sequences=1,
                            pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Sorgu Yanıt Sistemi
def answer_question(query):
    relevant_logs = find_relevant_logs(query)
    response = generate_response(relevant_logs)
    return response

# Model ve Tokenizer Yükleme
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

Sütun Adları: Index(['IP Address', 'Date', 'Request', 'Endpoint', 'Status Code',
       'Response Size', 'Referrer', 'User Agent', 'Time Taken'],
      dtype='object')
Eksik Veriler: IP Address       0
Date             0
Request          0
Endpoint         0
Status Code      0
Response Size    0
Referrer         0
User Agent       0
Time Taken       0
dtype: int64
0                 GET /usr/admin 502
1                POST /usr/login 500
2    DELETE /usr/admin/developer 200
3       PUT /usr/admin/developer 200
4           DELETE /usr/register 502
Name: Combined, dtype: object
0


C:\Users\reyiz\anaconda3\envs\NLP\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
#Test Sorgusu
query = "Son 24 saatte hangi URL'ler 200 hatası aldı?"  # Buraya sorgunuzu girin
response = answer_question(query)
print("Modelin Yanıtı:", response)

Modelin Yanıtı: 2019-02-23 10:21:21+03:00 DELETE /usr 200 2019-05-02 03:05:48+03:00 DELETE /usr 200 2018-08-27 10:40:48+03:00 DELETE /usr 200 2019-04-06 06:07:04+03:00 DELETE /usr 200 2018-07-09 04:59:51+03:00 DELETE /usr 200 2018-07-09 04:59:51+03:00 DELETE /usr 200 2018-07-09 04:59:51+03:00 DELETE /usr 200 2018-07-09 04:59:51
